# Pair Programming ETL Transformación III

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import ast

## Cargamos el `csv` en el que se trabajará

In [2]:
df = pd.read_csv("files/1.paises_meteo_info.csv", index_col = 0)
df.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",25,-1,12,1010,none,1,0,105,3,Papua New Guinea
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",26,-1,12,1012,none,1,0,85,3,Papua New Guinea


In [3]:
df2 = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col =0)
df2.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [4]:
df.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'pais'],
      dtype='object')

## Definición de variables

In [5]:
# Definimos las variables que vamos a necesitar para hacer las 
# llamadas a cada uno de los métodos de la clase.

dic_paises = {"USA":[-100.445882,39.7837304,], 
            "Australia":[134.755,-24.7761086],
            "South_Africa":[24.991639,-28.8166236],
            "New Zealand":[172.8344077,-41.5000831],
            "Papua New Guinea":[144.2489081,-5.6816069]}
producto_meteo = "meteo"
df_empty = pd.DataFrame()
name_file_wind = "3.clima.csv"
name_file_rh = "files/4.clima_wind.csv"

## Creamos la clase 

In [6]:
class Extraccion: 
    # definimos el método constructor con las variables globales que usaremos.
    def __init__(self, diccionario_paises, dataframe_ataques): # Recibe un diccionario con los países y coordenadas, el dataframe del archivo araques de tiburones y en formato string el nombre de la columna "layer"
        
        self.diccionario_paises = diccionario_paises
        self.dataframe_ataques = dataframe_ataques
        self.columna = "layer"
        
    # El método llamará a la API para cada país deseado

    def llamada_api (self, dataframe_vacio, producto): # recibe un dataframe vacío y el nombre del producto a solicitar a la api

        # Definimos la variable 
        self.dataframe_vacio = dataframe_vacio
        self.producto = producto

        # hacemos la llamada  a la API iterando por cada uno de los países
        for k,v in self.diccionario_paises.items():
            lon = (v[0])
            lat = (v[1])

            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        
        response = requests.get(url=url)
        
        codigo_estado = response.status_code
        
        razon_estado = response.reason
        
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        
        response.json()
        # Se convierten los datos obtenidos en un df
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        
        # Creamos una columna llamada país donde se vaya registrando cada país
        df["country"] = k

        # Concatenamos los df que hemos obtenido con cada país
        df_final = pd.concat([self.dataframe_vacio,df],axis=0, ignore_index= True)

        # Se guarda el df obtenido con la información de la API
        df_final.to_csv("files/3.clima.csv")

        return df_final

    # Este método filtra el df sobre ataques de tiburones a los países deseados
    def filtrar_por_paises (self): #no recibe parámetro
        
        df_ataques_tib = self.dataframe_ataques[(self.dataframe_ataques["country"]== 'usa') | 
            (self.dataframe_ataques["country"]== "australia") | 
            (self.dataframe_ataques["country"]== "new zealand") |
            (self.dataframe_ataques["country"]== "south africa") |
            (self.dataframe_ataques["country"]== "papua new guinea")]
    
        return df_ataques_tib

    # Este método limpia la columna 'wind_profile' del dataframe obtenido a través de la API
    def limpiar_columnas_wind (self, nombre_archivo_wind):

        self.nombre_archivo_wind = nombre_archivo_wind

        # Se abre el archivo que contiene la información obtenida de la API
        df3 = pd.read_csv(f"files/{self.nombre_archivo_wind}")

        df3.reset_index()

        df3['wind_profile']= df3['wind_profile'].apply(ast.literal_eval) # Se requiere el método ast.literal_eval para convertirlo a lista, 
                                                                        #si previamente ha sido guardado como csv, ya que pierde esa propiedad
                                                                        #Si se trabaja con un arhivo que no ha sido guardado como csv, no haría 
                                                                        # falta el ast.literal_eval

        z = df3['wind_profile'].apply(pd.Series)
        z.head()

        # Por eso empezamos con un for para iterar por cada una de las columnas. 
        for i in range(len(z.columns)): 

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "wind_dir_" + str(z[i].apply(pd.Series)[self.columna][0]) 

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(z[i].apply(pd.Series)['direction'])

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df3.insert(i, nombre, valores)

        df3.to_csv("files/4.clima_wind.csv")

        return df3

    # Este método limpia la columna 'rh_profile' del dataframe obtenido a través de la API
    def limpiar_columnas_rh (self,nombre_archivo_rh):
        
        self.nombre_archivo_rh = nombre_archivo_rh

        # Se abre el archivo creado con la limpieza de la columna wind_profile
        df4 = pd.read_csv(self.nombre_archivo_rh)

        df4.reset_index()

        df4['rh_profile']= df4['rh_profile'].apply(ast.literal_eval) # Se requiere el método ast.literal_eval para convertirlo a lista, 
                                                                     #si previamente ha sido guardado como csv, ya que pierde esa propiedad
                                                                     #Si se trabaja con un arhivo que no ha sido guardado como csv, no haría 
                                                                     # falta el ast.literal_eval

        z = df4['rh_profile'].apply(pd.Series)
        z.head()

        # Itera por cada una de las columnas. 
        for i in range(len(z.columns)): 

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "rh_" + str(z[i].apply(pd.Series)[self.columna][0]) 

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(z[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df4.insert(i, nombre, valores)

            # Guardamos el dataframe obtenido con la limpieza de las columnas.
            df4.to_csv("files/5.clima_wind_rh.csv")

        return df4

    

## Aplicación de la clase Extracción

In [7]:
# Creamos una variable llamada api que es resultado de la aplicación de la 
# clase Extracción, recibe como argumentos latitud, longitud y ciudad (Variables definidas anteriormente).

api = Extraccion(dic_paises,df2)
api

In [8]:
# Se aplica el método llamada api y se define un dataframe
df_country = api.llamada_api (df_empty, producto_meteo)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [9]:
# Se comprueba que se hayan cargado los datos correctamente
df_country.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",25,-1,12,1010,none,1,0,105,3,Papua New Guinea
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",26,-1,12,1012,none,1,0,85,3,Papua New Guinea


In [10]:
#Se analiza a la columna rh_profile
df_country["rh_profile"][0]


[{'layer': '950mb', 'rh': 15},
 {'layer': '900mb', 'rh': 12},
 {'layer': '850mb', 'rh': 10},
 {'layer': '800mb', 'rh': 9},
 {'layer': '750mb', 'rh': 1},
 {'layer': '700mb', 'rh': -2},
 {'layer': '650mb', 'rh': -3},
 {'layer': '600mb', 'rh': -3},
 {'layer': '550mb', 'rh': -2},
 {'layer': '500mb', 'rh': -3},
 {'layer': '450mb', 'rh': -3},
 {'layer': '400mb', 'rh': -3},
 {'layer': '350mb', 'rh': -3},
 {'layer': '300mb', 'rh': -2},
 {'layer': '250mb', 'rh': -2},
 {'layer': '200mb', 'rh': 0}]

In [11]:
type(df_country["rh_profile"][0])

list

In [12]:
#Se aplica el método filtrar por países para obtener un dataframe con los países que nos interesan
df_ataques_filtrado = api.filtrar_por_paises()

In [13]:
# Se comprueba que se hayan cargado los datos correctamente
df_ataques_filtrado.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [14]:
df_ataques_filtrado["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [15]:
#Se aplica el método limpiar columnas wind para limpiar la columna wind_profile

df_wind_limpio = api.limpiar_columnas_wind (name_file_wind)

In [16]:
# Se comprueba que se hayan cargado los datos correctamente
df_wind_limpio.head(2)

,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,85,85,85,85,70,70,90,280,275,295,...,25,-1,12,1010,none,1,0,105,3,Papua New Guinea
1,85,85,90,85,75,85,130,245,290,290,...,26,-1,12,1012,none,1,0,85,3,Papua New Guinea


In [17]:
# Se aplica el método para limpiar la columna rh
df_rh_limpio = api.limpiar_columnas_rh (name_file_rh)

In [18]:
# Se comprueba que se hayan cargado los datos correctamente
df_rh_limpio.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,15,12,10,9,1,-2,-3,-3,-2,-3,...,25,-1,12,1010,none,1,0,105,3,Papua New Guinea
1,14,11,8,6,1,-2,-3,-4,-3,-3,...,26,-1,12,1012,none,1,0,85,3,Papua New Guinea
